In [ ]:
import sys
import os
sys.path.append("/home/smets/codes/far/PHARE/pyphare/")
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
import numpy as np
%matplotlib widget

In [ ]:
run_path = "/DATA/phare/ionBeam/run01/01a"
os.listdir(run_path)

In [ ]:
run  = Run(run_path)
time = 10.0

**Get the `main` hierarchy (the other population `beam` is not considered here)**

In [ ]:
main_hier = run.GetParticles(time, "main")

**Get the domain box to eventually create sub-box to locally define the pressure**

In [ ]:
domain_box = main_hier.domain_box

**Get the interpolator for each B component (for velocity projection) with `merged=True` option**

In [ ]:
Bx, xy = run.GetB(time, merged=True, interp='nearest')['Bx']
By, xy = run.GetB(time, merged=True, interp='nearest')['By']
Bz, xy = run.GetB(time, merged=True, interp='nearest')['Bz']

**Create the `p_parallel` and `p_perp` arrays on L0 level**

In [ ]:
n0 = (run.GetDomainSize()/run.GetDl()).astype(int)

#num_of_part = np.zeros(n0)
#v_parallel  = np.zeros(n0)
#p_parallel  = np.zeros(n0)
#v_perp      = np.zeros(n0)
#p_perp      = np.zeros(n0)

num_of_part = np.ones(n0)
v_parallel  = np.ones(n0)
p_parallel  = np.ones(n0)
v_perp      = np.ones(n0)
p_perp      = np.ones(n0)

**Iteration through the particle hierarchy to get the particle velocity (parallel and perp)**

In [ ]:
for ilvl, lvl in main_hier.levels(time).items():
    for patch in lvl.patches:
        mpi_rank, patch_id = patch.id.strip('p').split('#')
        #print("mpi rank = {0} , patch id = {1}".format(mpi_rank, patch_id))
        pdata = patch.patch_datas['main_domain']
        icells = pdata.dataset.iCells
        uvws = pdata.dataset.v
        # this loop is quite slow... but interpolators do not accept arrays as entry
        for index in np.arange(icells.shape[0]):
            i, j = icells[index,0], icells[index,1]
            B = np.asarray([Bx(i, j), By(i, j), Bz(i, j)])
            bb = np.sqrt(B.dot(B))
            b = B/bb
            v = np.asarray([uvws[index,0], uvws[index,1], uvws[index,2]])
            v_parallel[i, j] = np.dot(v, b)
            vp_              = np.cross(v, b)
            v_perp[i, j]     = np.sqrt(vp_.dot(vp_))
            num_of_part[i, j] += 1
            #break
v_parallel = v_parallel/num_of_part
v_perp = v_perp/num_of_part

In [ ]:
v_parallel.max()

**Then, 2D-image of the parallel velocity**

In [ ]:
x = np.linspace(0, run.GetDomainSize()[0], n0[0]+1)
y = np.linspace(0, run.GetDomainSize()[1], n0[1]+1)

xv, yv = np.meshgrid(x, y, indexing = 'ij')

fig, ax = plt.subplots()

im = ax.pcolormesh(xv, yv, v_parallel, cmap = 'viridis')
plt.colorbar(im)

In [ ]:
print(n0[0])
print(x)